In [1]:
import pandas as pd
from sklearn.ensemble import HistGradientBoostingRegressor

In [2]:


data = pd.read_csv("https://raw.githubusercontent.com/Garve/datasets/main/causal_data.csv")
data.head()

,x1,x2,x3,t,y
0,-0.652660,-0.713868,-1.122252,1,-4.394130
1,0.833597,0.303656,1.268176,0,2.267590
2,-0.366262,0.610887,1.239072,0,-1.705938
3,-1.446081,-0.480858,0.406920,0,-5.917297
4,-0.043417,-0.802488,0.088580,0,-2.839481


In [5]:
data.t.value_counts()

t
0    69892
1    30108
Name: count, dtype: int64

### S-learner

In [3]:

model = HistGradientBoostingRegressor()
X = data.drop(["y"], axis=1)
y = data["y"]

model.fit(X, y)

HistGradientBoostingRegressor()

In [7]:
X["t"] = 1
X_all_treatment = X.copy()
X_all_treatment.t.value_counts()

t
1    100000
Name: count, dtype: int64

In [8]:
X["t"] = 0
X_all_control = X.copy()

treatment_effects = model.predict(X_all_treatment) - model.predict(X_all_control)

print(treatment_effects)

[-3.16738908e-02  4.00288522e+00  4.14661974e+00 ...  3.75600554e+00
 -2.08010497e-03 -8.26595053e-03]


In [9]:
print(treatment_effects.mean())

1.9950775036539583


You will find out that the average treatment effect is around 2. So, if you would treat every individual in your dataset, your outcome would increase by around 2 on average compared to if nobody gets any treatment

### T-learner

In [10]:
model_0 = HistGradientBoostingRegressor()
model_1 = HistGradientBoostingRegressor()

data_0 = data[data.t == 0]
data_1 = data[data.t == 1]

X_0 = data_0[["x1", "x2", "x3"]]
y_0 = data_0[["y"]]
X_1 = data_1[["x1", "x2", "x3"]]
y_1 = data_1[["y"]]

model_0.fit(X_0, y_0)
model_1.fit(X_1, y_1)

data_without_treatment = data[["x1", "x2", "x3"]]
treatment_effects = model_1.predict(data_without_treatment) - model_0.predict(data_without_treatment)

/Users/keithlowton/anaconda3/envs/Causal/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/keithlowton/anaconda3/envs/Causal/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
